In [1]:
import sys
#!{sys.executable} -m pip install opencv-contrib-python
#!{sys.executable} -m pip install --upgrade opencv-contrib-python
!pip install opencv-contrib-python


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import cv2
import numpy as np
from cv2 import aruco

# Load the original image and the replacement image
input_img = "./lroom/00/0000000150.jpg"
# Define the ID of the marker you want to replace
target_marker_id = 3  # Change this to the ID of the marker you want to target
replacement_img_path = "./demo/lamp_rep.jpg"

input_img = "./lroom/00/0000000207.jpg"
# Define the ID of the marker you want to replace
target_marker_id = 4  # Change this to the ID of the marker you want to target
replacement_img_path = "./demo/lamp_rep.jpg"

input_img = "./lroom/00/0000000176.jpg"
# Define the ID of the marker you want to replace
target_marker_id = 46  # Change this to the ID of the marker you want to target
replacement_img_path = "./demo/lamp_rep.jpg"

# Resize the replacement image to be larger than the marker
replacement_size_factor = 3 # Factor to enlarge the replacement image

original_image = cv2.imread(input_img)
replacement_image = cv2.imread(replacement_img_path)

# Load the ArUco dictionary and parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters()



# Detect ArUco markers in the image
corners, ids, _ = aruco.detectMarkers(original_image, aruco_dict, parameters=parameters)

print(ids)

# Check if any markers were detected
if ids is not None and target_marker_id in ids:
    # Get the index of the target marker
    index = np.where(ids.flatten() == target_marker_id)[0][0]
    corner = corners[index][0]  # Get the corners of the target marker

    # Calculate the center of the detected marker
    center_x = int(np.mean(corner[:, 0]))
    center_y = int(np.mean(corner[:, 1]))
    center_point = (center_x, center_y)
    
    # Print the center point for verification
    print(f"Center point of marker ID {target_marker_id}: {center_point}")


    marker_width = int(np.linalg.norm(corner[0] - corner[1]))  # Calculate width of the marker
    marker_height = int(np.linalg.norm(corner[0] - corner[3]))  # Calculate height of the marker
    new_replacement_width = marker_width * replacement_size_factor
    new_replacement_height = marker_height * replacement_size_factor

    # Resize the replacement image
    replacement_image_resized = cv2.resize(replacement_image, (new_replacement_width, new_replacement_height))

    # Calculate the top-left corner for overlaying the replacement image
    top_left_x = center_x - new_replacement_width // 2
    top_left_y = center_y - new_replacement_height // 2

    # Overlay the resized replacement image onto the original image
    # Ensure the overlay is within the image boundaries
    h, w = original_image.shape[:2]
    if top_left_x < 0:
        top_left_x = 0
    if top_left_y < 0:
        top_left_y = 0
    if top_left_x + new_replacement_width > w:
        new_replacement_width = w - top_left_x
    if top_left_y + new_replacement_height > h:
        new_replacement_height = h - top_left_y

    # Create a region of interest (ROI) for the overlay
    roi = original_image[top_left_y:top_left_y + new_replacement_height, top_left_x:top_left_x + new_replacement_width]

    # Overlay the replacement image on the ROI
    replacement_image_resized_cropped = replacement_image_resized[0:new_replacement_height, 0:new_replacement_width]
    combined_roi = cv2.addWeighted(roi, 0, replacement_image_resized_cropped, 1, 0)  # No blending needed

    # Place the combined ROI back into the original image
    original_image[top_left_y:top_left_y + new_replacement_height, top_left_x:top_left_x + new_replacement_width] = combined_roi

    # Show the final image with the replacement
    cv2.imshow('Final Image', original_image)
    cv2.waitKey(0)
else:
    print(f"Marker ID {target_marker_id} not detected.")

cv2.destroyAllWindows()


[[35]
 [12]
 [19]
 [11]
 [31]
 [30]
 [ 6]
 [ 5]
 [40]
 [46]
 [ 4]
 [ 3]
 [26]
 [38]]
Center point of marker ID 46: (817, 605)


2024-11-01 18:43:00.761 Python[61412:4460653] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
